Project 1

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
#API use and web scrapping
import requests
from bs4 import BeautifulSoup
import re

In [2]:
sqlite_path = 'data/raw_data_project_m1.db'
conn_str = f'sqlite:///{sqlite_path}'
engine = create_engine(conn_str)

career_info 

In [3]:
query = 'SELECT * FROM career_info;'

In [4]:
career_info = pd.read_sql_query(query, engine)
career_info

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f
...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a


Normalized Job Titles

In [5]:
list_api = []
list_job_code = set(career_info['normalized_job_code'])

In [6]:
for i in list_job_code:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    result_jobs = response.json()
    list_api.append(result_jobs)
    
data_jobs = pd.DataFrame(list_api)


country_info

In [7]:
query2 = 'SELECT * FROM country_info;'
country_info = pd.read_sql_query(query2, engine)
country_info.head()

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city


personal_info

In [8]:
query3 = 'SELECT * FROM personal_info;'
personal_info = pd.read_sql_query(query3, engine)

poll_info

In [9]:
query4 = 'SELECT * FROM poll_info;'
poll_info = pd.read_sql_query(query4, engine)

API

In [10]:
response = requests.get('http://api.dataatwork.org/v1/jobs')

json_data = response.json()

In [11]:
job_info = pd.DataFrame(json_data)

Web sacrapping

In [12]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')
country_codes = soup.find_all('td')

In [13]:
list_country_codes =[c.text.strip().split('\n') for c in country_codes]


In [14]:
list_country_codes2 = [[l for l in sub if len(l)>1] for sub in list_country_codes]
list_country_codes3 = [l for sub in list_country_codes2 for l in sub]

In [15]:
codes_raw = []
countries= []
for i in list_country_codes3:
    if len(i)<5 or i == '(XK[1])' or i == '(CN_X_HK)':
        codes_raw.append(i)
    else:
        countries.append(i)

In [16]:
codes_str = ''.join(codes_raw)
codes = re.sub(r'[^a-zA-Z ]', ' ', codes_str).split()

In [17]:
codes[57]= 'CN_X_HK'
del codes[58]
del codes[58]

MERGED DF

In [18]:
codes_df = pd.DataFrame(codes)
countries_df = pd.DataFrame(countries)
country_codes_info = pd.concat([codes_df, countries_df.reindex(codes_df.index)], axis=1)
country_codes_info.columns = ['country_code', 'country']
country_codes_info.head()

,country_code,country
0,BE,Belgium
1,EL,Greece
2,LT,Lithuania
3,PT,Portugal
4,BG,Bulgaria


In [19]:
df_merge_countries = pd.merge(country_codes_info,
                 country_info,
                 on='country_code')
df_merge_countries = df_merge_countries[['uuid','country_code','country', 'rural']]
df_merge_countries

,uuid,country_code,country,rural
0,dcab3aa0-dc79-0133-412b-0a81e8b09a82,BE,Belgium,urban
1,2fb4da70-dcef-0133-cafd-0a81e8b09a82,BE,Belgium,urban
2,73771070-d8ec-0133-ae53-0a81e8b09a82,BE,Belgium,urban
3,3aa979b0-d93d-0133-c00b-0a81e8b09a82,BE,Belgium,rural
4,11c6e300-deb8-0133-5b60-0a81e8b09a82,BE,Belgium,urban
...,...,...,...,...
8220,a1ceab50-da22-0133-ea30-0a81e8b09a82,PL,Poland,urban
8221,74fec3c0-db5b-0133-dab4-0a81e8b09a82,PL,Poland,urban
8222,f8dc4450-da13-0133-97bb-0a81e8b09a82,PL,Poland,Non-Rural
8223,91b54950-d911-0133-c958-0a81e8b09a82,PL,Poland,city


In [20]:
data_jobs.head()
 

,uuid,title,normalized_job_title,parent_uuid,error
0,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,325e2251e20170928426473156bd3c48,NaN
1,7bb15eefb19ed09c59349baad7e8469e,Database Coordinator,database coordinator,b90ca4df5690002377a7b0f1f3d40781,NaN
2,775190277a849cba701b306a7b374c0a,Data Officer,data officer,596226d5f9ef63de41a852826d483d19,NaN
3,6bf4290b983d58ce153af610008846c9,Data Entry Technician,data entry technician,bf88358c56bb6cbe7eabec38ae333d19,NaN
4,fcaf4aac737e53ad24b9f32f08b8740f,Computer or Data Processing Systems Consultant,computer or data processing systems consultant,0148f61d4227497728ce33490843d056,NaN


In [21]:
df_merge_jobs = pd.merge(data_jobs, career_info, how= 'inner', left_on= ['uuid'], right_on = ['normalized_job_code'])


In [23]:
df_merge_jobs['quantity'] = df_merge_jobs.groupby('normalized_job_code')['normalized_job_code'].transform('count')
df_merge_jobs['percentage'] = (100 * (df_merge_jobs['quantity']/df_merge_jobs['quantity'].sum())).round(6).astype(str)+'%'
df_merge_jobs

,uuid_x,title,normalized_job_title,parent_uuid,error,uuid_y,dem_education_level,dem_full_time_job,normalized_job_code,quantity,percentage
0,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,325e2251e20170928426473156bd3c48,NaN,8fd95b00-da49-0133-fd8c-0a81e8b09a82,high,yes,559a21f836c93876f31b60e6d10656a7,11.0,0.003117%
1,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,325e2251e20170928426473156bd3c48,NaN,9ac4bfb0-dccc-0133-f893-0a81e8b09a82,low,yes,559a21f836c93876f31b60e6d10656a7,11.0,0.003117%
2,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,325e2251e20170928426473156bd3c48,NaN,69819860-d99e-0133-96f4-0a81e8b09a82,None,yes,559a21f836c93876f31b60e6d10656a7,11.0,0.003117%
3,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,325e2251e20170928426473156bd3c48,NaN,47f45fd0-dd5c-0133-49ce-0a81e8b09a82,medium,yes,559a21f836c93876f31b60e6d10656a7,11.0,0.003117%
4,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,325e2251e20170928426473156bd3c48,NaN,bf9996b0-da57-0133-f1e3-0a81e8b09a82,low,yes,559a21f836c93876f31b60e6d10656a7,11.0,0.003117%
...,...,...,...,...,...,...,...,...,...,...,...
9644,f36c6ada3a808453a0bf241c74a4e994,Reference Data Expert,reference data expert,c80b795c61b66ce7d93000761d21632f,NaN,3fbfdfe0-da81-0133-3795-0a81e8b09a82,medium,yes,f36c6ada3a808453a0bf241c74a4e994,9.0,0.00255%
9645,f36c6ada3a808453a0bf241c74a4e994,Reference Data Expert,reference data expert,c80b795c61b66ce7d93000761d21632f,NaN,c04cbf30-d8e9-0133-9a83-0a81e8b09a82,high,yes,f36c6ada3a808453a0bf241c74a4e994,9.0,0.00255%
9646,f36c6ada3a808453a0bf241c74a4e994,Reference Data Expert,reference data expert,c80b795c61b66ce7d93000761d21632f,NaN,01d0af00-d8e2-0133-b50a-0a81e8b09a82,high,yes,f36c6ada3a808453a0bf241c74a4e994,9.0,0.00255%
9647,f36c6ada3a808453a0bf241c74a4e994,Reference Data Expert,reference data expert,c80b795c61b66ce7d93000761d21632f,NaN,eb126470-d982-0133-78a5-0a81e8b09a82,low,yes,f36c6ada3a808453a0bf241c74a4e994,9.0,0.00255%


In [25]:
df_raw = pd.merge(df_merge_countries, df_merge_jobs, left_on = 'uuid', right_on = 'uuid_y')

In [26]:
df_raw.drop(['uuid_x', 'parent_uuid', 'error', 'uuid_y', 'dem_education_level', 'dem_full_time_job', 'normalized_job_title', 'normalized_job_code'], axis = 1, inplace = True)

In [28]:
df_study = df_raw[df_raw['title'].notna()]

In [29]:
df_study

,uuid,country_code,country,rural,title,quantity,percentage
2,73771070-d8ec-0133-ae53-0a81e8b09a82,BE,Belgium,urban,Data Security Administrator,67.0,0.018987%
3,3aa979b0-d93d-0133-c00b-0a81e8b09a82,BE,Belgium,rural,Data Warehouse Developer,62.0,0.01757%
4,11c6e300-deb8-0133-5b60-0a81e8b09a82,BE,Belgium,urban,Geographic Information Systems Data Manager (G...,64.0,0.018137%
5,ed7e4bd0-d987-0133-3f9e-0a81e8b09a82,BE,Belgium,rural,Health Data Analyst,13.0,0.003684%
6,b5d05310-db4b-0133-8b2f-0a81e8b09a82,BE,Belgium,rural,Director of Data Operations,78.0,0.022104%
...,...,...,...,...,...,...,...
8216,51b2f400-d9ac-0133-2031-0a81e8b09a82,PL,Poland,urban,Database Architect,72.0,0.020404%
8217,314e9840-d982-0133-74f3-0a81e8b09a82,PL,Poland,urban,Database Administrator (DBA),10.0,0.002834%
8220,a1ceab50-da22-0133-ea30-0a81e8b09a82,PL,Poland,urban,Data Storage Specialist,83.0,0.023521%
8221,74fec3c0-db5b-0133-dab4-0a81e8b09a82,PL,Poland,urban,Automatic Data Processing Planner,78.0,0.022104%
